<h2>Comparison of EC, RS and their components</h2>

<h2>1. Import package, data, and define function</h2>

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import ee
import geemap

from statsmodels.tools.eval_measures import rmse

# Authenticate and Initialize EE
ee.Authenticate()
ee.Initialize()

<h2>2. Pre-processing in GEE</h2>

In [3]:
Tower_location = ee.Geometry.Point([115.71365, -31.37641])

# Add map
Map = geemap.Map()

# Shapefile
Tower_whole_shp = './Shapefile/Tower_footprint.shp'
Tower_whole = geemap.shp_to_ee(Tower_whole_shp)

# Arguments to filter the image collection
start_date = '2011-10-01'
startDate = ee.Date.parse('YYYY-MM-dd',start_date)

end_date = '2022-11-01'
endDate = ee.Date.parse('YYYY-MM-dd',end_date)

ls_cloud_cover = 15

# GENERAL FUNCTION

# Clip function
def clip_fnc(image):
    return image.clip(Tower_whole.geometry())

# Count band function
def count_band_fnc(image):
    band_no_var = image.bandNames().size()
    return image.copyProperties(image).set({'band_no':band_no_var})

# Count properties function
def count_property_fnc(image):
    property_no_var = image.propertyNames().size()
    return image.set({'property_no':property_no_var})

# Extract bit function
def bitWiseExtract(input, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return input.rightShift(fromBit).bitwiseAnd(mask)

# Pixel area function
def pixel_area_fnc(image):
    pixel_area_var = ee.Image.pixelArea()
    image_to_use_var = image.select(0)
    image_to_use_mask = image_to_use_var.gt(0)
    image_to_use_area = pixel_area_var.updateMask(image_to_use_mask)
    area_var = image_to_use_area.reduceRegion(reducer = ee.Reducer.sum(), geometry = Tower_whole.geometry())
    return image.set({'pixel_area':area_var.get('area')})

In [3]:
# CMRSET
# Define QA function
def cmrset_QA_fnc(image):
    cmrset_qa_var = image.select('pixel_qa')
    cmrset_mask_var = bitWiseExtract(cmrset_qa_var,0,1).eq(3)
    return image.updateMask(cmrset_mask_var)

# Filter collection
cmrset = (ee.ImageCollection('TERN/AET/CMRSET_LANDSAT_V2_2')
          .filterBounds(Tower_whole.geometry())
          .filterDate(start_date,end_date)
          .select([0,1],["ET","pixel_qa"])
          .map(cmrset_QA_fnc)
          .map(lambda image: image.set({'ET_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('ET')}))
          .map(count_property_fnc)
          .filterMetadata('property_no','equals',10))

# Generate a dataframe
cmrset_df = pd.DataFrame(list(zip(cmrset.aggregate_array('system:time_start').getInfo(), cmrset.aggregate_array('ET_value').getInfo())), columns=['Datetime','ET'])

# LANDSAT SURFACE REFLECTANCE

# Define QA function
def l57_QA_fnc(image):
    l57_qa_var = image.select('pixel_qa')
    l57_value1_var = l57_qa_var.eq(5440) #CLEAR, LOW SET
    l57_value2_var = l57_qa_var.eq(5504) #WATER, LOW SET
    l57_mask_var = l57_value1_var.Or(l57_value2_var)
    return image.updateMask(l57_mask_var)

def l89_QA_fnc(image):
    l89_qa_var = image.select('pixel_qa')
    l89_value1_var = l89_qa_var.eq(21824) #CLEAR, LOW SET
    l89_value2_var = l89_qa_var.eq(21888) #WATER, LOW SET
    l89_mask_var = l89_value1_var.Or(l89_value2_var)
    return image.updateMask(l89_mask_var)

# Define spectral indice calculation
def ls_fexp_spec_ind(image):

    # Normalized Difference Vegetation Index
    ndvi =  image.normalizedDifference(['NIR', 'R']).rename('NDVI')    
    
    # Enhanced Vegetation Index
    evi = image.expression('2.5 * ((N - R) / (N + (6 * R) - (7.5 * B) + 1))', {
        'N': image.select('NIR').multiply(0.0000275).subtract(0.2),
        'R': image.select('R').multiply(0.0000275).subtract(0.2),
        'B': image.select('B').multiply(0.0000275).subtract(0.2),}).rename('EVI')
    
    # Global Vegetation Moisture Index 
    gvmi = image.expression('((N + 0.1) - (S + 0.02))/((N + 0.1) + (S + 0.02))', {
        'N': image.select('NIR').multiply(0.0000275).subtract(0.2),
        'S': image.select('SWIR_1').multiply(0.0000275).subtract(0.2),}).rename('GVMI')
    
    # Soil Adjusted Vegetation Index (SAVI) 
    savi = image.expression(
      '((1 + 0.5)*(B5 - B4)) / (0.5 + (B5 + B4))', {
        'B4': image.select('R').multiply(0.0000275).subtract(0.2),
        'B5': image.select('NIR').multiply(0.0000275).subtract(0.2),
    }).rename('SAVI')
    
    # Normalised Difference Water Index (NDWI)
    ndwi =  image.normalizedDifference(['GR', 'NIR']).rename('NDWI')
    savi1 = savi.where(savi.gt(0.689), 0.689)
    
    # Leaf Area Index (LAI)
    lai = image.expression('-(log(( 0.69-SAVI)/0.59 ) / 0.91)', {'SAVI': savi1}).rename('LAI')

    NDVI_adjust = (ndvi.clamp(0.0, 1.00))
    fipar = (NDVI_adjust.multiply(1).subtract(ee.Number(0.05))).rename('fipar')
    fipar = fipar.clamp(0,1)

    # Broad-band Surface Emissivity (e_0)
    e_0 = image.expression('0.95 + 0.01 * LAI',{'LAI': lai})
    e_0 = e_0.where(lai.gt(3), 0.98).rename('e_0')

    # Narrow Band Transmissivity (e_NB)
    e_NB = image.expression('0.97 + (0.0033 * LAI)',{'LAI': lai})
    e_NB = e_NB.where(lai.gt(3), 0.98).rename('e_NB')
    log_eNB = e_NB.log()

    # Land Surface Temperature (LST) [K]
    comp_onda = ee.Number(1.115e-05)
    lst = image.expression('Tb / ( 1+ ( ( comp_onda * Tb / fator) * log_eNB))',{
        'Tb': image.select('BRT').multiply(0.00341802).add(149),
        'comp_onda': comp_onda,
        'log_eNB': log_eNB,
        'fator': ee.Number(1.438e-02),
      }).rename('LST')

    # Rescaled Brightness Temperature
    brt_r = image.select('BRT').multiply(0.00341802).add(149).rename('BRT_R')
    
    return image.addBands([ndvi, evi, gvmi, savi, ndwi, lst, lai, e_0, e_NB, brt_r])

# Filter collection
ls5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
       .filterBounds(Tower_whole.geometry())
       .filterDate(start_date,end_date)
       .filterMetadata('CLOUD_COVER','less_than',ls_cloud_cover)
       .select([0,1,2,3,4,8,5,17],["B","GR","R","NIR","SWIR_1","BRT","SWIR_2","pixel_qa"])
       .map(l57_QA_fnc)
       .map(ls_fexp_spec_ind)
       .map(lambda image: image.set({
           'NDVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDVI'),
           'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('EVI'),
           'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('GVMI'),
           'SAVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('SAVI'),
           'NDWI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDWI'),
           'LST_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LST'),
           'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LAI'),
           'e_0_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_0'),
           'e_NB_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_NB'),
           'BRT_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('BRT')}))
       .map(count_property_fnc)
       .filterMetadata('property_no','equals',118))

ls7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
       .filterBounds(Tower_whole.geometry())
       .filterDate(start_date,end_date)
       .filterMetadata('CLOUD_COVER','less_than',ls_cloud_cover)
       .select([0,1,2,3,4,8,5,17],["B","GR","R","NIR","SWIR_1","BRT","SWIR_2","pixel_qa"])
       .map(l57_QA_fnc)
       .map(ls_fexp_spec_ind)
       .map(lambda image: image.set({
           'NDVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDVI'),
           'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('EVI'),
           'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('GVMI'),
           'SAVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('SAVI'),
           'NDWI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDWI'),
           'LST_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LST'),
           'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LAI'),
           'e_0_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_0'),
           'e_NB_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_NB'),
           'BRT_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('BRT')}))
       .map(count_property_fnc)
       .filterMetadata('property_no','equals',144))

ls8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
       .filterBounds(Tower_whole.geometry())
       .filterDate(start_date,end_date)
       .filterMetadata('CLOUD_COVER','less_than',ls_cloud_cover)
       .select([0,1,2,3,4,5,6,8,17],["UB","B","GR","R","NIR","SWIR_1","SWIR_2","BRT","pixel_qa"])
       .map(l89_QA_fnc)
       .map(ls_fexp_spec_ind)
       .map(lambda image: image.set({
           'NDVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDVI'),
           'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('EVI'),
           'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('GVMI'),
           'SAVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('SAVI'),
           'NDWI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDWI'),
           'LST_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LST'),
           'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LAI'),
           'e_0_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_0'),
           'e_NB_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_NB'),
           'BRT_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('BRT')}))
       .map(count_property_fnc)
       .filterMetadata('property_no','equals',108))

ls9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
       .filterBounds(Tower_whole.geometry())
       .filterDate(start_date,end_date)
       .filterMetadata('CLOUD_COVER','less_than',ls_cloud_cover)
       .select([0,1,2,3,4,5,6,8,17],["UB","B","GR","R","NIR","SWIR_1","SWIR_2","BRT","pixel_qa"])
       .map(l89_QA_fnc)
       .map(ls_fexp_spec_ind)
       .map(lambda image: image.set({
           'NDVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDVI'),
           'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('EVI'),
           'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('GVMI'),
           'SAVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('SAVI'),
           'NDWI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('NDWI'),
           'LST_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LST'),
           'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('LAI'),
           'e_0_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_0'),
           'e_NB_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('e_NB'),
           'BRT_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 30, maxPixels = 1e9).get('BRT')}))
       .map(count_property_fnc)
       .filterMetadata('property_no','equals',104))

# Merge collection and generate a dataframe
ls = ls5.merge(ls7).merge(ls8).merge(ls9).sort("system:time_start")

# Generate a dataframe
ls_df = pd.DataFrame(list(zip(ls.aggregate_array('system:time_start').getInfo(), 
                              ls.aggregate_array('NDVI_value').getInfo(), ls.aggregate_array('EVI_value').getInfo(), 
                              ls.aggregate_array('GVMI_value').getInfo(), ls.aggregate_array('SAVI_value').getInfo(), 
                              ls.aggregate_array('NDWI_value').getInfo(), ls.aggregate_array('LST_value').getInfo(),
                              ls.aggregate_array('LAI_value').getInfo(), ls.aggregate_array('e_0_value').getInfo(),
                              ls.aggregate_array('e_NB_value').getInfo(), ls.aggregate_array('BRT_value').getInfo())), 
                     columns = ['Datetime','NDVI','EVI','GVMI','SAVI','NDWI','LST','LAI','e_0','e_NB','BRT'])

# MODIS Nadir BRDF-Adjusted Reflectance Daily 500m (MCD43A4) & MODIS Terra Surface Reflectance 8-day Global 500m (MOD09A1) & VIIRS Surface Reflectance Daily 500m and 1km (VNP09GA)

# Define QA function
def mcd43a4_QA_fnc(image):
    mcd34a4_value1_var = image.select('QA1').eq(0)
    mcd34a4_value2_var = image.select('QA2').eq(0)
    mcd34a4_value3_var = image.select('QA3').eq(0)
    mcd34a4_value4_var = image.select('QA4').eq(0)
    mcd34a4_value5_var = image.select('QA5').eq(0)
    mcd34a4_value6_var = image.select('QA6').eq(0)
    mcd34a4_value7_var = image.select('QA7').eq(0)
    mcd43a4_mask_var = (mcd34a4_value1_var.add(mcd34a4_value2_var).add(mcd34a4_value2_var).add(mcd34a4_value2_var).add(mcd34a4_value2_var).add(mcd34a4_value2_var).add(mcd34a4_value2_var))
    return image.updateMask(mcd43a4_mask_var)

def mod09a1_QA_fnc(image):
    mod09a1_qa_var = image.select('pixel_qa')
    mod09a1_mask_01_var = bitWiseExtract(mod09a1_qa_var,0,1).eq(0)
    mod09a1_mask_25_var = bitWiseExtract(mod09a1_qa_var,2,5).eq(0)
    mod09a1_mask_69_var = bitWiseExtract(mod09a1_qa_var,6,9).eq(0)
    mod09a1_mask_1013_var = bitWiseExtract(mod09a1_qa_var,10,13).eq(0)
    mod09a1_mask_1417_var = bitWiseExtract(mod09a1_qa_var,14,17).eq(0)
    mod09a1_mask_1821_var = bitWiseExtract(mod09a1_qa_var,18,21).eq(0)
    mod09a1_mask_2225_var = bitWiseExtract(mod09a1_qa_var,22,25).eq(0)
    mod09a1_mask_2629_var = bitWiseExtract(mod09a1_qa_var,26,29).eq(0)
    mod09a1_mask_30_var = bitWiseExtract(mod09a1_qa_var,30,30).eq(1)
    mod09a1_mask_31_var = bitWiseExtract(mod09a1_qa_var,6,9).eq(1)
    mod09a1_mask_var = (mod09a1_mask_01_var.add(mod09a1_mask_25_var).add(mod09a1_mask_69_var).add(mod09a1_mask_1013_var).add(mod09a1_mask_1417_var).add(mod09a1_mask_1821_var)
                       .add(mod09a1_mask_2225_var).add(mod09a1_mask_2629_var).add(mod09a1_mask_30_var).add(mod09a1_mask_31_var))
    return image.updateMask(mod09a1_mask_var)

def vnp09ga_QA_fnc(image):
    vnp09ga_mask_qf5_var = image.select('QF5')
    vnp09ga_mask_qf5_3_var = bitWiseExtract(vnp09ga_mask_qf5_var,3,3).eq(0)
    vnp09ga_mask_qf5_4_var = bitWiseExtract(vnp09ga_mask_qf5_var,4,4).eq(0)
    vnp09ga_mask_qf5_5_var = bitWiseExtract(vnp09ga_mask_qf5_var,5,5).eq(0)
    vnp09ga_mask_qf5_6_var = bitWiseExtract(vnp09ga_mask_qf5_var,6,6).eq(0)
    vnp09ga_mask_qf5_7_var = bitWiseExtract(vnp09ga_mask_qf5_var,7,7).eq(0)
    
    vnp09ga_mask_qf6_var = image.select('QF6')
    vnp09ga_mask_qf6_1_var = bitWiseExtract(vnp09ga_mask_qf6_var,1,1).eq(0)
    vnp09ga_mask_qf6_2_var = bitWiseExtract(vnp09ga_mask_qf6_var,2,2).eq(0) 
    
    vnp09ga_mask_var = (vnp09ga_mask_qf5_3_var.add(vnp09ga_mask_qf5_4_var).add(vnp09ga_mask_qf5_5_var).add(vnp09ga_mask_qf5_6_var)
                        .add(vnp09ga_mask_qf5_7_var).add(vnp09ga_mask_qf6_1_var).add(vnp09ga_mask_qf6_2_var))
    return image.updateMask(vnp09ga_mask_var)

# Define spectral indice calculation
def modis_viirs_fexp_spec_ind(image):
    
    # Enhanced Vegetation Index
    evi = image.expression('2.5 * ((N - R) / (N + (6 * R) - (7.5 * B) + 1))', {
        'N': image.select('NIR').multiply(0.0001),
        'R': image.select('R').multiply(0.0001),
        'B': image.select('B').multiply(0.0001),}).rename('EVI')
    
    # Global Vegetation Moisture Index 
    gvmi = image.expression('((N + 0.1) - (S + 0.02))/((N + 0.1) + (S + 0.02))', {
        'N': image.select('NIR').multiply(0.0001),
        'S': image.select('SWIR_1').multiply(0.0001),}).rename('GVMI')
    
    image = image.addBands([evi, gvmi])
    return image

# Filter collections
mcd43a4 = (ee.ImageCollection('MODIS/061/MCD43A4')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select([0,1,2,3,5,6,7,8,9,10,11,12,13],["R","NIR","B","GR","SWIR_1","SWIR_2","QA1","QA2","QA3","QA4","QA5","QA6","QA7"])
           .map(mcd43a4_QA_fnc)
           .map(modis_viirs_fexp_spec_ind)
           .map(lambda image: image.set({'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('EVI'),
                                         'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('GVMI')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',12))

mod09a1 = (ee.ImageCollection('MODIS/061/MOD09A1')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select([0,1,2,3,5,6,7],["R","NIR","B","G","SWIR_1","SWIR_2","pixel_qa"])
           .map(mod09a1_QA_fnc)
           .map(modis_viirs_fexp_spec_ind)
           .map(lambda image: image.set({'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('EVI'),
                                         'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('GVMI')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',12))

vnp09ga = (ee.ImageCollection('NOAA/VIIRS/001/VNP09GA')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select([1,2,3,4,5,7,8,26,27],["UB","B","G","R","NIR","SWIR_1","SWIR_2","QF5","QF6"])
           .map(vnp09ga_QA_fnc)
           .map(modis_viirs_fexp_spec_ind)
           .map(lambda image: image.set({'EVI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 1000, maxPixels = 1e9).get('EVI'),
                                         'GVMI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 1000, maxPixels = 1e9).get('GVMI')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',12))

# Generate a dataframe
mcd43a4_df = pd.DataFrame(list(zip(mcd43a4.aggregate_array('system:time_start').getInfo(), 
                                   mcd43a4.aggregate_array('EVI_value').getInfo(), 
                                   mcd43a4.aggregate_array('GVMI_value').getInfo())), 
                          columns=['Datetime','EVI','GVMI'])

mod09a1_df = pd.DataFrame(list(zip(mod09a1.aggregate_array('system:time_start').getInfo(), 
                                   mod09a1.aggregate_array('EVI_value').getInfo(), 
                                   mod09a1.aggregate_array('GVMI_value').getInfo())), 
                          columns=['Datetime','EVI','GVMI'])

vnp09ga_df = pd.DataFrame(list(zip(vnp09ga.aggregate_array('system:time_start').getInfo(),
                                   vnp09ga.aggregate_array('EVI_value').getInfo(), 
                                   vnp09ga.aggregate_array('GVMI_value').getInfo())), 
                          columns=['Datetime','EVI','GVMI'])

# AUSTRALIAN GRIDDED CLIMATE DATA (AGCD)

# Filter collection
agcd = (ee.ImageCollection('projects/tern-landscapes/AGCD/Daily')
        .filterBounds(Tower_whole.geometry())
        .filterDate(start_date,end_date)
        .select('rain','tmax','tmin','vp9am','vp3pm','solar')
        .map(lambda image: image.set({'rain_value':image.reduceRegion(reducer = ee.Reducer.sum(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('rain'),
                                      'tmax_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('tmax'),
                                      'tmin_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('tmin'),
                                      'vp9am_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('vp9am'),
                                      'vp3pm_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('vp3pm'),
                                      'solar_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('solar')}))
        .map(count_property_fnc)
        .filterMetadata('property_no','equals',41))

# Generate a dataframe
agcd_df = pd.DataFrame(list(zip(agcd.aggregate_array('system:time_start').getInfo(), 
                                agcd.aggregate_array('rain_value').getInfo(), agcd.aggregate_array('tmax_value').getInfo(), 
                                agcd.aggregate_array('tmin_value').getInfo(), agcd.aggregate_array('vp9am_value').getInfo(), 
                                agcd.aggregate_array('vp3pm_value').getInfo(), agcd.aggregate_array('solar_value').getInfo())), 
                       columns=['Datetime','Rain','Ta_max','Ta_min','VPD_9am','VPD_3pm','Solar'])

In [4]:
# MODIS Land Cover Type Yearly Global 500m (MCD12Q1) for both MOD16 and PML
mod12q1 = (ee.ImageCollection('MODIS/061/MCD12Q1'))

In [5]:
# MOD16A2 (use MOD16A2.006 rather than MOD16A2.061 as the newer only covers from 2021) 

# Define QA function
def mod16a2_QA_fnc(image):
    mod16a2_qa_var = image.select('pixel_qa')
    mod16a2_mask_qc_var = bitWiseExtract(mod16a2_qa_var,0,0).eq(0)
    mod16a2_mask_sensor_var = bitWiseExtract(mod16a2_qa_var,1,1).eq(0)
    mod16a2_mask_cloud_state_var = bitWiseExtract(mod16a2_qa_var,3,4).eq(0)
    mod16a2_mask_confidence_var = bitWiseExtract(mod16a2_qa_var,5,7).lte(1)
    mod16a2_mask_var = mod16a2_mask_qc_var.add(mod16a2_mask_sensor_var).add(mod16a2_mask_cloud_state_var).add(mod16a2_mask_confidence_var)
    return image.updateMask(mod16a2_mask_var)

# Define conversion calculation
def mod16a2_convert(image):
    
    # Converted ET
    et_converted = image.expression('(ET*0.1)/8',{'ET':image.select('ET')}).rename('ET_converted')
    
    image = image.addBands(et_converted)
    return image

# Filter collection 
mod16a2 = (ee.ImageCollection('MODIS/006/MOD16A2')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select([0,4],["ET","pixel_qa"])
           .map(mod16a2_QA_fnc)
           .map(mod16a2_convert)
           .map(lambda image: image.set({'ET_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('ET_converted')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',11))

# Generate a dataframe
mod16a2_df = pd.DataFrame(list(zip(mod16a2.aggregate_array('system:time_start').getInfo(), mod16a2.aggregate_array('ET_value').getInfo())), columns=['Datetime','ET'])

# MODIS Terra Leaf Area Index/FPAR 8-Day Global 500m (MOD15A2)

# Define QA function 
def mod15a2_QA_fnc(image):
    mod15a2_qa_var = image.select('pixel_qa')
    mod15a2_mask_cloud_state_var = bitWiseExtract(mod15a2_qa_var,3,4).eq(0)
    mod15a2_mask_confidence_var = bitWiseExtract(mod15a2_qa_var,5,7).lte(1)
    mod15a2_mask_var = mod15a2_mask_cloud_state_var.add(mod15a2_mask_confidence_var)
    return image.updateMask(mod15a2_mask_var)

# Filter collection
mod15a2 = (ee.ImageCollection('MODIS/061/MOD15A2H')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select([0,1,2],["FPAR","LAI","pixel_qa"])
           .map(mod15a2_QA_fnc)
           .map(lambda image: image.set({'FPAR_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('FPAR'),
                                         'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('LAI')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',12))

# Generate a dataframe
mod15a2_df = pd.DataFrame(list(zip(mod15a2.aggregate_array('system:time_start').getInfo(), 
                                   mod15a2.aggregate_array('FPAR_value').getInfo(), 
                                   mod15a2.aggregate_array('LAI_value').getInfo())), 
                          columns=['Datetime','FPAR','LAI'])

# MODIS BRDF/Albedo Daily L3 0.05 Deg CMG (MCD43C3)

# Define QA function
def mcd43c3_QA_fnc(image):
    mcd43c3_qa_var = image.select('pixel_qa')
    mcd43c3_mask_var = bitWiseExtract(mcd43c3_qa_var,0,2).lte(2)
    return image.updateMask(mcd43c3_mask_var)

# Filter collection
mcd43c3 = (ee.ImageCollection('MODIS/061/MCD43C3')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select(["Albedo_WSA_shortwave","Albedo_Quality"],["Albedo","pixel_qa"]) # not BRDF_Quality, it's Albedo_Quality 
           .map(mcd43c3_QA_fnc)
           .map(lambda image: image.set({'Albedo_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 5600, maxPixels = 1e9).get('Albedo')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',11))

# Generate a dataframe
mcd43c3_df = pd.DataFrame(list(zip(mcd43c3.aggregate_array('system:time_start').getInfo(), 
                                   mcd43c3.aggregate_array('Albedo_value').getInfo())), 
                          columns=['Datetime','Albedo'])

# Global Modeling and Assimilation Office (product's spatial resolution is too big compared to the tower footprint, hence switch reduceRegion to use Tower_location, rather than Tower_whole footprint)

# Filter collection
gmao = (ee.ImageCollection('NASA/GEOS-CF/v1/rpl/tavg1hr')
        .filterBounds(Tower_whole.geometry())
        .filterDate('2018-01-01','2023-01-01')
        .select(["PS","T","Q"],
                ["Pa","Ta","Rh"]))

# Loop to generate a daily image collection as well as dataframe (loop to reduce the computation load)
gmao_year = np.linspace(2018,2022,5,dtype=int).tolist()

for x in range(len(gmao_year)):
    
    # Define daily function
    def gmao_daily_fnc(day):
        start = gmao_startDate_x.advance(ee.Number(day).subtract(1),'day')
        end = gmao_startDate_x.advance(ee.Number(day),'day')
        filtered = gmao.filterDate(start,end).mean().set({'system:time_start':start.millis()})
        return filtered
    
    # Start date
    gmao_startDate_str_x = str(gmao_year[x])+'-01-01'
    gmao_startDate_x = ee.Date.parse('YYYY-MM-dd',gmao_startDate_str_x)
    
    # End date
    gmao_endDate_str_x = str(gmao_year[x]+1)+'-01-01'
    gmao_endDate_x = ee.Date.parse('YYYY-MM-dd',gmao_endDate_str_x)
    
    # Generate a daily image collection
    gmao_day_x = ee.List.sequence(1, gmao_endDate_x.difference(gmao_startDate_x,'days'), 1)
    gmao_daily_x = (ee.ImageCollection(gmao_day_x.map(gmao_daily_fnc).flatten())
                    .map(lambda image: image.set({'Pa_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27750, maxPixels = 1e9).get('Pa'),
                                                  'Ta_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27750, maxPixels = 1e9).get('Ta'),
                                                  'Rh_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27750, maxPixels = 1e9).get('Rh')})))
    
    # Generate a dataframe
    gmao_daily_df_x = pd.DataFrame(list(zip(gmao_daily_x.aggregate_array('system:time_start').getInfo(), 
                                            gmao_daily_x.aggregate_array('Pa_value').getInfo(), 
                                            gmao_daily_x.aggregate_array('Ta_value').getInfo(), 
                                            gmao_daily_x.aggregate_array('Rh_value').getInfo())), columns=['Datetime','Pa','Ta','Rh'])
    
    # Concatnate dataframes
    if x == 0:
        gmao_daily = gmao_daily_df_x
    else:
        gmao_daily = pd.concat([gmao_daily,gmao_daily_df_x])

In [6]:
# PML

# Filter collection 
pml = (ee.ImageCollection('CAS/IGSNRR/PML/V2_v017')
       .filterBounds(Tower_whole.geometry())
       .filterDate(start_date,end_date)
       .select(["Ec","Es","Ei"],["ETc","ETs","ETi"])
       .map(lambda image: image.set({'ETc_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('ETc'),
                                     'ETs_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('ETs'),
                                     'ETi_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('ETi')}))
       .map(count_property_fnc)
       .filterMetadata('property_no','equals',9))

# Generate a dataframe ('system:time_start' has error, use code editor)
pml_df = pd.DataFrame(list(zip(pml.aggregate_array('system:time_start').getInfo(), 
                               pml.aggregate_array('ETc_value').getInfo(), 
                               pml.aggregate_array('ETs_value').getInfo(), 
                               pml.aggregate_array('ETi_value').getInfo())), 
                      columns=['Datetime','ETc','ETs','ETi'])

# MODIS Leaf Area Index/FPAR 4-Day Global 500m (MCD15A3)

# Define QA function
def mcd15a3_QA_fnc(image):
    mcd15a3_qa_var = image.select("pixel_qa")
    mcd15a3_mask_qc_var = bitWiseExtract(mcd15a3_qa_var,0,0).eq(0)
    mcd15a3_mask_cloud_state_var = bitWiseExtract(mcd15a3_qa_var,3,4).eq(0)
    mcd15a3_mask_confidence_var = bitWiseExtract(mcd15a3_qa_var,5,7).lte(1)
    mcd15a3_mask_var = mcd15a3_mask_qc_var.add(mcd15a3_mask_cloud_state_var).add(mcd15a3_mask_confidence_var)
    return image.updateMask(mcd15a3_mask_var)

# Filter collection
mcd15a3 = (ee.ImageCollection('MODIS/061/MCD15A3H')
           .filterDate(start_date,end_date)
           .select([1,2],["LAI","pixel_qa"])
           .map(mcd15a3_QA_fnc)
           .map(lambda image: image.set({'LAI_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('LAI')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',11))

# Generate a dataframe
mcd15a3_df = pd.DataFrame(list(zip(mcd15a3.aggregate_array('system:time_start').getInfo(), 
                                   mcd15a3.aggregate_array('LAI_value').getInfo())), 
                          columns=['Datetime','LAI'])

# Global 500-m MODIS Black-sky Albedo (MCD43A3)

# Define QA function
def mcd43a3_QA_fnc(image):
    mcd43a3_qa_var = image.select('pixel_qa')
    mcd43a3_mask_var = bitWiseExtract(mcd43a3_qa_var,0,0).eq(0)
    return image.updateMask(mcd43a3_mask_var)

# Filter collection
mcd43a3 = (ee.ImageCollection('MODIS/061/MCD43A3')           
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select(["Albedo_WSA_shortwave","BRDF_Albedo_Band_Mandatory_Quality_shortwave"],["Albedo","pixel_qa"])
           .map(mcd43a3_QA_fnc)
           .map(lambda image: image.set({'Albedo_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 500, maxPixels = 1e9).get('Albedo')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',11))

# Generate a dataframe
mcd43a3_df = pd.DataFrame(list(zip(mcd43a3.aggregate_array('system:time_start').getInfo(), 
                                   mcd43a3.aggregate_array('Albedo_value').getInfo())), 
                          columns=['Datetime','Albedo'])

# Global 1-km MODIS LST/Emissivity (MOD11A2)

# Filter collection
mod11a2 = (ee.ImageCollection('MODIS/061/MOD11A2')
           .filterBounds(Tower_whole.geometry())
           .filterDate(start_date,end_date)
           .select(["Emis_31","Emis_32"])
           .map(lambda image: image.set({'Emis_31_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 1000, maxPixels = 1e9).get('Emis_31'),
                                         'Emis_32_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_whole.geometry(), scale = 1000, maxPixels = 1e9).get('Emis_32')}))
           .map(count_property_fnc)
           .filterMetadata('property_no','equals',12))

# Generate a dataframe
mod11a2_df = pd.DataFrame(list(zip(mod11a2.aggregate_array('system:time_start').getInfo(), 
                                   mod11a2.aggregate_array('Emis_31_value').getInfo(), 
                                   mod11a2.aggregate_array('Emis_32_value').getInfo())), 
                          columns=['Datetime','Emis_31','Emis_32'])

# Global Land Data Assimilation System (GLDAS)

# Filter collection
gldas = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
         .filterBounds(Tower_whole.geometry())
         .filterDate('2011-01-01','2023-01-01')
         .select(["Psurf_f_inst","Tair_f_inst","Wind_f_inst","Qair_f_inst","Rainf_f_tavg","LWdown_f_tavg","SWdown_f_tavg"],
                 ["Pa","Ta","Ws","Rh","Precip","Rl_down","Rs_down"]))

# Loop to generate a daily image collection as well as dataframe (loop to reduce the computation load)
gldas_year = np.linspace(2011,2022,12,dtype=int).tolist()

for x in range(len(gldas_year)):
    
    # Define daily function
    def gldas_daily_fnc(day):
        start = gldas_startDate_x.advance(ee.Number(day).subtract(1),'day')
        end = gldas_startDate_x.advance(ee.Number(day),'day')
        filtered = gldas.filterDate(start,end).mean().set({'system:time_start':start.millis()})
        return filtered
    
    # Start date
    gldas_startDate_str_x = str(gldas_year[x])+'-01-01'
    gldas_startDate_x = ee.Date.parse('YYYY-MM-dd',gldas_startDate_str_x)

    # End date
    gldas_endDate_str_x = str(gldas_year[x]+1)+'-01-01'
    gldas_endDate_x = ee.Date.parse('YYYY-MM-dd',gldas_endDate_str_x)
   
    # Generate a daily image collection
    gldas_day_x = ee.List.sequence(1,gldas_endDate_x.difference(gldas_startDate_x,'days'), 1)
    gldas_daily_x = (ee.ImageCollection(gldas_day_x.map(gldas_daily_fnc).flatten())
                     .map(lambda image: image.set({'Pa_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Pa'),
                                                   'Ta_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Ta'),
                                                   'Ws_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Ws'),
                                                   'Rh_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Rh'),
                                                   'Precip_value':image.reduceRegion(reducer = ee.Reducer.sum(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Precip'),
                                                   'Rl_down_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Rl_down'),
                                                   'Rs_down_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 27830, maxPixels = 1e9).get('Rs_down'),})))
    
    # Generate a dataframe
    gldas_daily_df_x = pd.DataFrame(list(zip(gldas_daily_x.aggregate_array('system:time_start').getInfo(), 
                                             gldas_daily_x.aggregate_array('Pa_value').getInfo(), gldas_daily_x.aggregate_array('Ta_value').getInfo(), 
                                             gldas_daily_x.aggregate_array('Ws_value').getInfo(), gldas_daily_x.aggregate_array('Rh_value').getInfo(), 
                                             gldas_daily_x.aggregate_array('Precip_value').getInfo(), gldas_daily_x.aggregate_array('Rl_down_value').getInfo(),
                                             gldas_daily_x.aggregate_array('Rs_down_value').getInfo())), columns=['Datetime','Pa','Ta','Ws','Rh','Precip','Rl_down','Rs_down'])
    
    # Concatnate dataframes
    if x == 0:
        gldas_daily = gldas_daily_df_x
    else:
        gldas_daily = pd.concat([gldas_daily,gldas_daily_df_x])

In [7]:
# SEBAL

# ERA5-Land

# Filter collection
era5_land = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
             .filterBounds(Tower_whole.geometry())
             .filterDate('2011-01-01','2023-01-01')
             .select(["surface_solar_radiation_downwards_hourly","potential_evaporation_hourly","temperature_2m","u_component_of_wind_10m","v_component_of_wind_10m","dewpoint_temperature_2m"],
                     ["Rs_down","PET","Ta","Ws_u","Ws_v","tdp"]))

# Loop to generate a daily image collection as well as dataframe (loop to reduce the computation load)
era5_land_year = np.linspace(2011,2022,12,dtype=int).tolist()

for x in range(len(era5_land_year)):
    
    # Define daily function
    def era5_land_daily_fnc(day):
        start = era5_land_startDate_x.advance(ee.Number(day).subtract(1),'day')
        end = era5_land_startDate_x.advance(ee.Number(day),'day')
        filtered = era5_land.filterDate(start,end).mean().set({'system:time_start':start.millis()})
        return filtered
    
    # Start date
    era5_land_startDate_str_x = str(era5_land_year[x])+'-01-01'
    era5_land_startDate_x = ee.Date.parse('YYYY-MM-dd',era5_land_startDate_str_x)
    
    # End date
    era5_land_endDate_str_x = str(era5_land_year[x]+1)+'-01-01'
    era5_land_endDate_x = ee.Date.parse('YYYY-MM-dd',era5_land_endDate_str_x)    
    
    # Generate a daily image collection
    era5_land_day_x = ee.List.sequence(1,era5_land_endDate_x.difference(era5_land_startDate_x,'days'), 1)
    era5_land_daily_x = (ee.ImageCollection(era5_land_day_x.map(era5_land_daily_fnc).flatten())
                         .map(lambda image: image.set({'Rs_down_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('Rs_down'),
                                                       'PET_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('PET'),
                                                       'Ta_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('Ta'),
                                                       'Ws_u_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('Ws_u'),
                                                       'Ws_v_value':image.reduceRegion(reducer = ee.Reducer.sum(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('Ws_v'),
                                                       'tdp_value':image.reduceRegion(reducer = ee.Reducer.mean(), geometry = Tower_location, scale = 11132, maxPixels = 1e9).get('tdp'),})))

    # Generate a dataframe
    era5_land_daily_df_x = pd.DataFrame(list(zip(era5_land_daily_x.aggregate_array('system:time_start').getInfo(),
                                                 era5_land_daily_x.aggregate_array('Rs_down_value').getInfo(), 
                                                 era5_land_daily_x.aggregate_array('PET_value').getInfo(), 
                                                 era5_land_daily_x.aggregate_array('Ta_value').getInfo(), 
                                                 era5_land_daily_x.aggregate_array('Ws_u_value').getInfo(), 
                                                 era5_land_daily_x.aggregate_array('Ws_v_value').getInfo(), 
                                                 era5_land_daily_x.aggregate_array('tdp_value').getInfo())), 
                                        columns=['Datetime','Rs_down','PET','Ta','Ws_u','Ws_v','tdp'])
    
    # Concatnate dataframes
    if x == 0:
        era5_land_daily = era5_land_daily_df_x
    else:
        era5_land_daily = pd.concat([era5_land_daily,era5_land_daily_df_x])

<h2>3. Linear regression analysis</h2>

In [ ]:
# Set up datetime index
ec = ec.set_index(pd.date_range(start="2011-10-01",end="2022-10-31",freq="MS",inclusive="left")).drop(columns=['year','month'])
cmrset = cmrset.set_index(pd.date_range(start="2011-10-01",end="2022-10-31",freq="MS",inclusive="left")).drop(columns=['Datetime'])
mod16a2 = mod16a2.set_index(pd.to_datetime(mod16a2['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
sebal = sebal.set_index(pd.date_range(start="2011-10-01",end="2022-10-31",freq="MS",inclusive="left")).drop(columns=['year','month'])
landsat_archive = landsat_archive.set_index(pd.to_datetime(landsat_archive['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
mod15a2 = mod15a2.set_index(pd.to_datetime(mod15a2['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
mcd15a3 = mcd15a3.set_index(pd.to_datetime(mcd15a3['Datetime'], unit='ms')).drop(['Unnamed: 0','Datetime'], axis=1).resample('MS').mean()
mcd43a3 = mcd43a3.set_index(pd.to_datetime(mcd43a3['Datetime'], unit='ms')).drop(['Unnamed: 0','Datetime'], axis=1).resample('MS').mean()
mod11a2 = mod11a2.set_index(pd.to_datetime(mod11a2['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
agcd = agcd.set_index(pd.to_datetime(agcd['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
gldas = gldas.set_index(pd.date_range(start="2011-01-01",end="2023-01-01",freq="D",inclusive="left")).drop(columns=['Datetime']).resample('MS').mean()['2011-10-01':'2022-10-31']
era5 = era5.set_index(pd.date_range(start="2011-01-01",end="2023-01-01",freq="D",inclusive="left")).drop(columns=['Datetime']).resample('MS').mean()['2011-10-01':'2022-10-31']

# Combine PML ET from canopy transpiration, evaporation from interception, and soil evaporation
pml = pml.set_index(pd.to_datetime(pml['Datetime'], unit='ms')).drop(['Datetime'], axis=1).resample('MS').mean()
pml['ET'] = pml['ETc'] + pml['ETs'] + pml['ETi']

# Select indices from products and concatnate df
et_comp = pd.concat([ec, cmrset, mod16a2, pml['ET'], sebal, landsat_archive[['NDVI','EVI','GVMI','SAVI','NDWI','LST']], 
                     mcd15a3, mod15a2['FPAR'], mcd43a3, mod11a2['Emis_32'], gldas[['Pa','Rl_down']], era5, agcd['Rain']], axis=1)

et_comp.columns = [['EC', 'CMRSET', 'MOD16', 'PML', 'SEBAL', 'Ls', 'Ls', 'Ls', 'Ls', 'Ls', 'Ls', 
                    'MCD15A3', 'MOD15A2', 'MCD43A3', 'MOD11A2', 'GLDAS', 'GLDAS', 'ERA5', 'ERA5', 
                    'ERA5', 'ERA5', 'ERA5', 'ERA5', 'AGCD'],['ET', 'ET', 'ET', 'ET', 'ET', 'NDVI', 
                    'EVI', 'GVMI', 'SAVI', 'NDWI', 'LST', 'LAI', 'FPAR', 'Albedo', 'Emissivity', 
                    'Pa', 'Rl_down', 'Rs_down', 'PET', 'Ta', 'Ws_u', 'Ws_v', 'tdp', 'Precip']]

# Pearson's R correlation
# Conditional formatting to detect multi collonearity (in case of removing variables, remove those having less correlation with EC ET)
et_comp_styled = et_comp.corr(method='pearson').pow(2).style.background_gradient(vmax = 0.8).highlight_null('red')

# Keep EC ET, CMRSET ET, MOD16 ET, PML ET, Ls EVI, Ls GVMI, Ls LST, MCD15A3 LAI, MOD15A2 FPAR,  MCD43A3 Albedo, MOD11A2 Emissivity, 
# GLDAS Pa, GLDAS Rl_down, ERA5-Land Rs_down, ERA5 PET, ERA5 Ta, ERA5 Ws_u, ERA5 Ws_v, ERA5 tdp, ERA5 Precip 
et_comp_reduced = et_comp[[('EC','ET'),('CMRSET','ET'),('MOD16','ET'),('PML','ET'),('SEBAL','ET'),('Ls','EVI'),('Ls','GVMI'),('Ls','LST'),
                          ('MCD15A3','LAI'),('MOD15A2','FPAR'),('MCD43A3','Albedo'),('MOD11A2','Emissivity'),('GLDAS','Pa'),('GLDAS','Rl_down'),
                          ('ERA5','Rs_down'),('ERA5','PET'),('ERA5','Ta'),('ERA5','Ws_u'),('ERA5','Ws_v'),('ERA5','tdp'),('AGCD','Precip')]]

# R2
et_comp_reduced_corr = et_comp_reduced.corr(method='pearson').pow(2)

# Plot
et_comp_reduced_corr_plot = pd.concat([et_comp_reduced_corr[('EC','ET')],
                                      (et_comp_reduced_corr[('EC','ET')] - et_comp_reduced_corr[('CMRSET','ET')]),
                                      (et_comp_reduced_corr[('EC','ET')] - et_comp_reduced_corr[('MOD16','ET')]),
                                      (et_comp_reduced_corr[('EC','ET')] - et_comp_reduced_corr[('PML','ET')]),
                                      (et_comp_reduced_corr[('EC','ET')] - et_comp_reduced_corr[('SEBAL','ET')])],
                                     axis=1).drop([('EC','ET'),('CMRSET','ET'),('MOD16','ET'),('PML','ET'),('SEBAL','ET')],axis=0).reset_index(level=0, drop=True)
et_comp_reduced_corr_plot.columns = [['EC','CMRSET', 'MOD16', 'PML', 'SEBAL'],['ET','ET','ET','ET','ET']]
et_comp_reduced_corr_plot = et_comp_reduced_corr_plot.sort_values(by=[('PML','ET')])
bar_plot_x_axis = np.arange(len(et_comp_reduced_corr_plot.index))

<h2>4. Plot</h2>

In [ ]:
# Combined plot
plt.figure(figsize=(24,12))
plt.subplot(2,1,1)
plt.bar(et_comp_reduced_corr_plot.index,et_comp_reduced_corr_plot[('EC','ET')],width=0.4,color='tab:blue')
plt.xticks(fontsize=17.5,rotation=45)
plt.yticks(fontsize=17.5)
plt.ylim(0,0.6)
plt.ylabel('R\u00b2',fontsize=22.5)
plt.xlabel('Remotely sensed variables',fontsize=22.5)
plt.text(-0.9,0.545,'(A)',fontsize=22.5,fontweight='bold')
plt.grid(axis='y',zorder=10,color='lightgray',linewidth=0.5)

plt.subplot(2,1,2)
plt.bar(bar_plot_x_axis-0.15,et_comp_reduced_corr_plot[('CMRSET','ET')],width=0.1,color='tab:purple',alpha=0.6)
plt.bar(bar_plot_x_axis-0.05,et_comp_reduced_corr_plot[('MOD16','ET')],width=0.1,color='tab:cyan',alpha=0.6)
plt.bar(bar_plot_x_axis+0.05,et_comp_reduced_corr_plot[('PML','ET')],width=0.1,color='tab:brown')
plt.bar(bar_plot_x_axis+0.15,et_comp_reduced_corr_plot[('SEBAL','ET')],width=0.1,color='tab:olive',alpha=0.6)
plt.xticks(np.arange(0,len(et_comp_reduced_corr_plot.index),1),fontsize=17.5,rotation=45)
plt.yticks(fontsize=17.5)
plt.ylabel('Residual R\u00b2',fontsize=22.5)
plt.text(-0.9,0.135,'(B)',fontsize=22.5,fontweight='bold')
plt.grid(axis='y',zorder=10,color='lightgray',linewidth=0.5)

# Export
plt.tight_layout()
plt.savefig('R2 between EC, RS and variables.png')